# Model Results Visualization
This notebook is intended to pull model artifacts from the Neptune experiment logger and run it on a few samples from the dataset to see visualize how the different model's are segmenting the data. 

In [3]:
from neptune import Session
import pickle
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [7]:
image_types = ['full_channel', 'rgb', 'ir', 'hsv', 'hsv_with_ir', 'veg_index']

In [8]:
f = open("image_samples.pkl", "rb")
image_samples = pickle.load(f)

In [10]:
# create session and import my project
sesh = Session.with_default_backend(api_token="eyJhcGlfYWRkcmVzcyI6Imh0dHBzOi8vdWkubmVwdHVuZS5haSIsImFwaV91cmwiOiJodHRwczovL3VpLm5"
                                "lcHR1bmUuYWkiLCJhcGlfa2V5IjoiOGE5NDI0YTktNmE2ZC00ZWZjLTlkMjAtNjNmMTIwM2Q2ZTQzIn0=")
project = sesh.get_project("maxzvyagin/GIS")

In [11]:
experiments = project.get_experiments()

In [12]:
experiments

[Experiment(GIS-130)]

In [ ]:
for e in experiments:
    # load in the model
    model = pickle.loads(e.get_artifacts()[0])
    # check the image type, and get image samples for corresponding image type
    i_type = e.get_parameters()['image_type']
    # run the model on each of the samples and show results
    # print experiment id, name, image type, and training/test loss
    print(e.get_properties(), e.get_numeric_channels_values()['test_loss'])
    # show mask, 